# Script Demo
Show how to use the provided scripts that help with creating expertiment designs and processing the results

## Creating Experiment Design

### Cross Product Experiment Design (from table format)
Converts an experiment config in `table` form (concise) into the experiment design form required for the experiment suite.     
The function builds a cartesian product of all configuration options marked as `$FACTOR$` with a list of levels.    
The experiment in table form is a yaml file (with name `<exp_name>.yml` in `experiments/table`).
Each factor of the experiment is a yaml object with a single entry with key `$FACTOR$` and as a value the list of levels.

See example `experiments/table/demo.yml`and the resulting `experiments/designs/demo.yml`.

In [53]:
#%pycat scripts/expdesign.py # uncomment to see the code
%run scripts/expdesign.py --suite demo --exps demo_exp1 demo_exp2

with open("experiments/designs/demo.yml", "r") as f:
    experiments_config = yaml.load(f, Loader=yaml.SafeLoader)
display(experiments_config)

Writing experiment design to: experiments/designs/demo.yml



{'demo_exp1': {'n_repetitions': 2,
  'common_roles': ['setup-common'],
  'host_types': {'client': {'n': 1, 'init_role': 'setup-client'},
   'server': {'n': 1, 'check_status': False, 'init_role': 'setup-server'}},
  'base_experiment': {'w': 13,
   'x': '$FACTOR$',
   'host_vars': {'client': {'y': 'abc'}, 'server': {'z': '$FACTOR$'}}},
  'factor_levels': [{'x': 4, 'host_vars': {'server': {'z': 1}}},
   {'x': 4, 'host_vars': {'server': {'z': 2}}},
   {'x': 5, 'host_vars': {'server': {'z': 1}}},
   {'x': 5, 'host_vars': {'server': {'z': 2}}},
   {'x': 6, 'host_vars': {'server': {'z': 1}}},
   {'x': 6, 'host_vars': {'server': {'z': 2}}},
   {'x': 7, 'host_vars': {'server': {'z': 1}}},
   {'x': 7, 'host_vars': {'server': {'z': 2}}}]},
 'demo_exp2': {'n_repetitions': 2,
  'common_roles': ['setup-common'],
  'host_types': {'client': {'n': 1, 'init_role': 'setup-client'},
   'server': {'n': 1, 'init_role': 'setup-server'}},
  'base_experiment': {'a': 10,
   'b': 'info',
   'c': '$FACTOR$',
   '

## Reading and Processing Results

In [48]:
import pandas as pd

results_dir = "results"

In [50]:
# A dict with all experiment names and for each a list of experiment ids that should be included in the dataframe.
# See `scripts/results.py` to adjust how different output files are used.  
#%pycat scripts/results.py # uncomment to see the code
%run scripts/results.py

exp = {
    "example" : ["1628751557"]
}

df = read_df(results_dir, exp)
display(df)

suite_name    suite_id      exp_name  run  rep host_type  host_idx info  \
0     example  1628751557  experiment_1    2    1    server         0   c3   
1     example  1628751557  experiment_1    2    1    client         0   c3   
2     example  1628751557  experiment_1    2    0    server         0   c3   
3     example  1628751557  experiment_1    2    0    client         0   c3   
4     example  1628751557  experiment_1    1    1    server         0   c2   
5     example  1628751557  experiment_1    1    1    client         0   c2   
6     example  1628751557  experiment_1    1    0    server         0   c2   
7     example  1628751557  experiment_1    1    0    client         0   c2   
8     example  1628751557  experiment_1    0    1    server         0   c1   
9     example  1628751557  experiment_1    0    1    client         0   c1   
10    example  1628751557  experiment_1    0    0    server         0   c1   
11    example  1628751557  experiment_1    0    0    client         0   c1   
12    example  1628751557  experiment_2    0    0    server         0   c2   
13    example  1628751557  experiment_2    0    0    client         0   c2   

    host_vars.client.arg1  host_vars.client.arg2  host_vars.server.arg3  \
0                      10                      3                      3   
1                      10                      3                      3   
2                      10                      3                      3   
3                      10                      3                      3   
4                      10                      2                      2   
5                      10                      2                      2   
6                      10                      2                      2   
7                      10                      2                      2   
8                      10                      1                      1   
9                      10                      1                      1   
10                     10                      1                      1   
11                     10                      1                      1   
12                     10                      2                      2   
13                     10                      2                      2   

   host_vars.server.arg4       a0           a1   a2  a3   a4     a5  tp  
0                  test1  -config  config.json  -a3   3  -a4  test1   6  
1                  test1  -config  config.json  -a1  10  -a2      3  17  
2                  test1  -config  config.json  -a3   3  -a4  test1  15  
3                  test1  -config  config.json  -a1  10  -a2      3   9  
4                  test1  -config  config.json  -a3   2  -a4  test1  16  
5                  test1  -config  config.json  -a1  10  -a2      2  21  
6                  test1  -config  config.json  -a3   2  -a4  test1   2  
7                  test1  -config  config.json  -a1  10  -a2      2  18  
8                  test1  -config  config.json  -a3   1  -a4  test1  15  
9                  test1  -config  config.json  -a1  10  -a2      1   1  
10                 test1  -config  config.json  -a3   1  -a4  test1  12  
11                 test1  -config  config.json  -a1  10  -a2      1  12  
12                 test1  -config  config.json  -a3   2  -a4  test1   9  
13                 test1  -config  config.json  -a1  10  -a2      2  11

### Group By (Aggregate Repetitions)
The experiment repeats each run `r` times.
Here we show how to apply this group by aggregate transformation on the dataframe to calculate mean, min, max, and std dev across repetitions of the same run.
(for clarity, we exclude the other config options but they could be used as further group by columns) 

In [51]:
# Example 1: Only keep the main info

df1 = df

# sometimes (e.g., for csv result files) it's necessary to explicitly specify which columns are numbers
df1[["host_vars.client.arg1", "tp"]] = df1[["host_vars.client.arg1", "tp"]].apply(pd.to_numeric)


df1 = df1.groupby(['exp_name', 'suite_id', 'run', 'host_type']).agg({'tp': ['mean', 'min', 'max', 'std']}).reset_index()

df1.columns = ["_".join(v) if v[1] else v[0] for v in df1.columns.values]

display(df1)

exp_name    suite_id  run host_type  tp_mean  tp_min  tp_max    tp_std
0  experiment_1  1628751557    0    client      6.5       1      12  7.778175
1  experiment_1  1628751557    0    server     13.5      12      15  2.121320
2  experiment_1  1628751557    1    client     19.5      18      21  2.121320
3  experiment_1  1628751557    1    server      9.0       2      16  9.899495
4  experiment_1  1628751557    2    client     13.0       9      17  5.656854
5  experiment_1  1628751557    2    server     10.5       6      15  6.363961
6  experiment_2  1628751557    0    client     11.0      11      11       NaN
7  experiment_2  1628751557    0    server      9.0       9       9       NaN

In [52]:
# Example 2: Keep all config info
cols = df.columns.tolist()
cols.remove('rep') # remove rep because we want to aggregate over the reps
cols.remove('tp') # remove the `value` column `tp` (throughput)

df1 = df.groupby(cols).agg({'tp': ['mean', 'min', 'max', 'std']}).reset_index()
df1.columns = ["_".join(v) if v[1] else v[0] for v in df1.columns.values]
display(df1)

suite_name    suite_id      exp_name  run host_type  host_idx info  \
0    example  1628751557  experiment_1    0    client         0   c1   
1    example  1628751557  experiment_1    0    server         0   c1   
2    example  1628751557  experiment_1    1    client         0   c2   
3    example  1628751557  experiment_1    1    server         0   c2   
4    example  1628751557  experiment_1    2    client         0   c3   
5    example  1628751557  experiment_1    2    server         0   c3   
6    example  1628751557  experiment_2    0    client         0   c2   
7    example  1628751557  experiment_2    0    server         0   c2   

   host_vars.client.arg1  host_vars.client.arg2  host_vars.server.arg3  ...  \
0                     10                      1                      1  ...   
1                     10                      1                      1  ...   
2                     10                      2                      2  ...   
3                     10                      2                      2  ...   
4                     10                      3                      3  ...   
5                     10                      3                      3  ...   
6                     10                      2                      2  ...   
7                     10                      2                      2  ...   

        a0           a1   a2  a3   a4     a5 tp_mean  tp_min  tp_max    tp_std  
0  -config  config.json  -a1  10  -a2      1     6.5       1      12  7.778175  
1  -config  config.json  -a3   1  -a4  test1    13.5      12      15  2.121320  
2  -config  config.json  -a1  10  -a2      2    19.5      18      21  2.121320  
3  -config  config.json  -a3   2  -a4  test1     9.0       2      16  9.899495  
4  -config  config.json  -a1  10  -a2      3    13.0       9      17  5.656854  
5  -config  config.json  -a3   3  -a4  test1    10.5       6      15  6.363961  
6  -config  config.json  -a1  10  -a2      2    11.0      11      11       NaN  
7  -config  config.json  -a3   2  -a4  test1     9.0       9       9       NaN  

[8 rows x 21 columns]